In [1]:
import pandas as pd
import sys
sys.path.append('../')

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-27/departures.csv')

In [3]:
from datasets.crsp_data import CRSP

In [5]:
crsp = CRSP()

DOWNLOADING RAW FILE
DOWNLOADING RAW FILE
